#Reload a fresh Keras model from the saved model

In [168]:
import IPython.display
import librosa
import librosa.display
import pandas as pd
import os
import struct
import glob
import soundfile as sf
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import specgram
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from keras.callbacks import ModelCheckpoint
from datetime import datetime
from sklearn import metrics
import librosa as lb
import IPython.display as ipd
from tensorflow import keras
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization
from tqdm import tqdm

seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

#PREPARE SCREAM DATASET

In [169]:
#DATASET USED
#https://www.kaggle.com/datasets/sanzidaakterarusha/scream-dataset?datasetId=1903423&sortBy=dateRun&tab=collaboration

# Se obtienen las diferentes rutas de los datos, tanto audios como metadata y path para obtener las muestras
file_path = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/UrbanSounds8k/UrbanSound8K/audio'
screams_path = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/sounds-screams/good'
urbansound8k = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/UrbanSounds8k/UrbanSound8K/metadata/UrbanSound8K.csv')
file_viz = glob.glob('/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/UrbanSounds8k/UrbanSound8K/audio/fold1/*')

In [170]:
#pd.set_option('display.max_rows', None)
urbansound8k.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


In [171]:
#Se organiza dataset para seleccionar la clase gun shot de interes y las otras renombrarse como no scream
urbansound8k.loc[urbansound8k["class"] != " ", "class"] = "non_scream"
#Se cambian los valores de la columna classID para identificar unicamente las dos clases de interes
urbansound8k.loc[urbansound8k["classID"] != 10, "classID"] = 0 #non_event
#se cuenta el numero total de muestras de la clase de interes
totalSamples = 0
lastNameFilesScreams=[]
for file in os.listdir(screams_path):
    if file.endswith('.wav'):
        lastNameFilesScreams.append(file)
        totalSamples+=1
print(totalSamples)
#se toman las muestras aleatorias del tamaño de las muestras de la clase de interes para que quede balanceado
urbansound8k[urbansound8k['class'] == "non_scream"].sample(n=totalSamples).head()
#se toman una las muestras aleatorias del tamaño de las muestras de la clase de interes para que quede balanceado
dfNonScream = urbansound8k[urbansound8k['class'] == "non_scream"].sample(n=totalSamples)
#Se crea dataframe con los datos de los audios de sonidos de interes
dfScream = pd.DataFrame(lastNameFilesScreams, columns=['slice_file_name'])
dfScream['classID'] = 1 #scream
dfScream['class'] = 'scream'
dfScream['fold'] = 0
dfScream
#se unen en un solo dataset las clases de interes con muestras balanceadas
dfCompleteScream = dfNonScream.append(dfScream)
dfCompleteScream.count()[0]
dfCompleteScream[dfCompleteScream['class'] == "scream"].count()[0]
#Se cambia el dataset para unicamente tomar las clases de interés para el trabajo
dfCompleteScream.head()

391


<ipython-input-171-a9de57edfae9>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfCompleteScream = dfNonScream.append(dfScream)


,slice_file_name,fsID,start,end,salience,fold,classID,class
2852,161129-4-0-11.wav,161129.0,13.775687,17.775687,1.0,8,0,non_scream
6135,34056-2-0-2.wav,34056.0,1.000000,5.000000,2.0,4,0,non_scream
1023,123685-5-0-8.wav,123685.0,7.845506,11.845506,1.0,7,0,non_scream
2481,157867-8-0-10.wav,157867.0,5.000000,9.000000,1.0,1,0,non_scream
1967,14772-7-9-0.wav,14772.0,31.891222,35.786547,1.0,7,0,non_scream


In [172]:
#Se define funcion para extrar las caracteristicas con la librería librosa, obetiendo los coeficientes ceptrales de frecuencia de Mel
#Se realiza un pading en el tamaño para que concuerden los tamaños de las caracteristicas de entrada al modelo.(?)
def extract_features(file_name, Nmfcc, Nfft, NhopL, NwinL):

    samplerate = 22050
    longitudMaxAudio = 4
    max_pad_len = int(samplerate*longitudMaxAudio/NhopL) + int(samplerate*longitudMaxAudio/NhopL*0.05)  #Calculo longitud de salida de mfcc con 5% de tolerancia para longitud de audios

    try:
      audio, sample_rate = librosa.load(file_name, res_type='soxr_hq')
      mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=Nmfcc, n_fft=Nfft, hop_length=NhopL, win_length=NwinL)
      pad_width = max_pad_len - mfccs.shape[1]
      mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')

    except Exception as e:
      print("Error encountered while parsing file: ", file_name)
      return None
    #print(mfccs.shape)
    return mfccs

In [173]:
#Se realiza la extracción de caracteristicas, teniendo en cuenta la clase, si el sonido es de la carpeta agregada de la clase explosions va y busca este sonido en la carpeta requerida

def get_features(Nmfcc, Nfft, NhopL, NwinL):
  features = []
  # Iterate through each sound file and extract the features
  for index, row in dfCompleteScream.iterrows():
      if(row["class"]=="non_scream"):
          file_name = os.path.join(os.path.abspath(file_path),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
          #file_name = ""
      elif(row["class"]=="scream"):
          file_name = os.path.join(os.path.abspath(screams_path),str(row["slice_file_name"]))
          #file_name = ""
      #print(file_name)
      class_label = row["classID"]
      data = extract_features(file_name, Nmfcc, Nfft, NhopL, NwinL)

      features.append([data, class_label])

  # Convert into a Panda dataframe
  featuresdf = pd.DataFrame(features, columns=['feature','class_label'])
  return featuresdf

In [174]:
def splitFeaturesTrainTest(featuresdf, num_rows, num_columns, num_channels):
  X = np.array(featuresdf.feature.tolist())
  y = np.array(featuresdf.class_label.tolist())

  # Encode the classification labels
  le = LabelEncoder()
  yy = to_categorical(le.fit_transform(y))

  # split the dataset
  from sklearn.model_selection import train_test_split

  x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state = 3)

  x_train = x_train.reshape(x_train.shape[0], num_rows, num_columns, num_channels)
  x_test = x_test.reshape(x_test.shape[0], num_rows, num_columns, num_channels)
  num_labels = yy.shape[1]
  return x_train, x_test, y_train, y_test, num_labels

In [175]:


#1	3	4096	4096	4096	0	2
#2	12	4096	4096	4096	0	3
#3	27	4096	4096	4096	0	3
#4	45	4096	4096	4096	0	5


samplerate = 22050
longitudMaxAudio = 4
Nmfcc = 3
Nfft = 4096
NwinL = 4096
iterableNhopL = 1.0
NhopL =  4096       #int(iterableNhopL*NwinL)
k_size = 2
num_rows = Nmfcc
num_columns = int(samplerate*longitudMaxAudio/NhopL) + int(samplerate*longitudMaxAudio/NhopL*0.05)  #Calculo longitud de salida de mfcc con 5% de tolerancia para longitud de audios
num_channels = 1


featuresdfScream = get_features(Nmfcc, Nfft, NhopL, NwinL)
x_trainScream, x_testScream, y_trainScream, y_testScream, num_labelsScream = splitFeaturesTrainTest(featuresdfScream, num_rows, num_columns, num_channels)


#PREPARE GUNSHOT DATASET

In [176]:
# Se obtienen las diferentes rutas de los datos, tanto audios como metadata y path para obtener las muestras
file_path = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/UrbanSounds8k/UrbanSound8K/audio'
explosions_path = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/sounds-explosions/explosions'
urbansound8k = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/UrbanSounds8k/UrbanSound8K/metadata/UrbanSound8K.csv')
file_viz = glob.glob('/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/UrbanSounds8k/UrbanSound8K/audio/fold1/*')

In [177]:
#Se organiza dataset para seleccionar la clase gun shot de interes y las otras renombrarse como no gunshot
urbansound8k.loc[urbansound8k["class"] != "gun_shot", "class"] = "non_gun_shot"
#Se cambian los valores de la columna classID para identificar unicamente las dos clases de interes
urbansound8k.loc[urbansound8k["classID"] != 6, "classID"] = 0  #non_event
urbansound8k.loc[urbansound8k["classID"] == 6, "classID"] = 1  #gun_shot

#se cuenta el numero total de muestras de la clase de interes
totalSamplesGunshot =  urbansound8k.loc[((urbansound8k["class"]=="gun_shot"))].count()[0]
totalSamplesGunshot

#se carga el numero total de muestras de la clase a agregar al dataset - explosions
totalSamplesExplosions = 0
lastNameFilesExplosions=[]
for file in os.listdir(explosions_path):
    if file.endswith('.wav'):
        lastNameFilesExplosions.append(file)
        totalSamplesExplosions+=1
print(totalSamplesExplosions)

#Se crea dataframe con los datos de los audios de sonidos de la clase a agregar - explosions
dfExplosion = pd.DataFrame(lastNameFilesExplosions, columns=['slice_file_name'])
dfExplosion['classID'] = 0 #non_event
dfExplosion['class'] = 'non_gun_shot'
dfExplosion['fold'] = 0

#Agregar en un dataset todos los sonidos de interes con la clase nueva agregada - explosions
print(urbansound8k.count()[0])
urbansound8k = urbansound8k.append(dfExplosion)
urbansound8k.count()[0]

#se toma una muestra aleatoria del tamaño de las muestras de la clase de interes para que quede balanceado
dfNonGunShot = urbansound8k[urbansound8k['class'] == "non_gun_shot"].sample(n=totalSamplesGunshot)
dfGunShot= urbansound8k[urbansound8k['class'] == "gun_shot"]

#se unen en un solo dataset las clases de interes con muestras balanceadas
dfCompleteGunshot = dfNonGunShot.append(dfGunShot)
dfCompleteGunshot.count()[0]

209
8732


<ipython-input-177-5260589046a1>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  urbansound8k = urbansound8k.append(dfExplosion)
<ipython-input-177-5260589046a1>:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfCompleteGunshot = dfNonGunShot.append(dfGunShot)


748

In [178]:
#Se define funcion para extrar las caracteristicas con la librería librosa, obetiendo los coeficientes ceptrales de frecuencia de Mel
#Se realiza un pading en el tamaño para que concuerden los tamaños de las caracteristicas de entrada al modelo.

def extract_features(file_name, Nmfcc, Nfft, NhopL, NwinL):

    samplerate = 22050
    longitudMaxAudio = 4
    max_pad_len = int(samplerate*longitudMaxAudio/NhopL) + int(samplerate*longitudMaxAudio/NhopL*0.05)  #Calculo longitud de salida de mfcc con 5% de tolerancia para longitud de audios

    try:
      audio, sample_rate = librosa.load(file_name, res_type='soxr_hq')
      mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=Nmfcc, n_fft=Nfft, hop_length=NhopL, win_length=NwinL)
      pad_width = max_pad_len - mfccs.shape[1]
      mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')

    except Exception as e:
      print("Error encountered while parsing file: ", file_name)
      return None
    #print(mfccs.shape)
    return mfccs

In [179]:
#Se realiza la extracción de caracteristicas, teniendo en cuenta la clase, si el sonido es de la carpeta agregada de la clase explosions va y busca este sonido en la carpeta requerida

def get_features(Nmfcc, Nfft, NhopL, NwinL):
  features = []
  # Iterate through each sound file and extract the features
  for index, row in dfCompleteGunshot.iterrows():
      if(row["fold"]!=0):
          file_name = os.path.join(os.path.abspath(file_path),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
          #file_name = ""
      elif(row["fold"]==0):
          file_name = os.path.join(os.path.abspath(explosions_path),str(row["slice_file_name"]))
          #file_name = ""
      #print(file_name)
      class_label = row["classID"]
      data = extract_features(file_name, Nmfcc, Nfft, NhopL, NwinL)

      features.append([data, class_label])

  # Convert into a Panda dataframe
  featuresdf = pd.DataFrame(features, columns=['feature','class_label'])
  return featuresdf

In [180]:
def splitFeaturesTrainTest(featuresdf, num_rows, num_columns, num_channels):
  X = np.array(featuresdf.feature.tolist())
  y = np.array(featuresdf.class_label.tolist())

  # Encode the classification labels
  le = LabelEncoder()
  yy = to_categorical(le.fit_transform(y))

  # split the dataset
  from sklearn.model_selection import train_test_split

  x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state = 3)

  x_train = x_train.reshape(x_train.shape[0], num_rows, num_columns, num_channels)
  x_test = x_test.reshape(x_test.shape[0], num_rows, num_columns, num_channels)
  num_labels = yy.shape[1]
  return x_train, x_test, y_train, y_test, num_labels

In [181]:

#1	3	4096	4096	4096	0	2
#2	12	4096	4096	4096	0	3
#3	27	4096	4096	4096	0	3
#4	45	4096	4096	4096	0	5

samplerate = 22050
longitudMaxAudio = 4
Nmfcc = 3
Nfft = 4096
NwinL = 4096
iterableNhopL = 1.0
NhopL =  4096       #int(iterableNhopL*NwinL)
k_size = 2
num_rows = Nmfcc
num_columns = int(samplerate*longitudMaxAudio/NhopL) + int(samplerate*longitudMaxAudio/NhopL*0.05)  #Calculo longitud de salida de mfcc con 5% de tolerancia para longitud de audios
num_channels = 1


featuresdfGunshot = get_features(Nmfcc, Nfft, NhopL, NwinL)
x_trainGunshot, x_testGunshot, y_trainGunshot, y_testGunshot, num_labelsGunshot = splitFeaturesTrainTest(featuresdfGunshot, num_rows, num_columns, num_channels)

/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=4096 is too large for input signal of length=3969
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=4096 is too large for input signal of length=3949
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=4096 is too large for input signal of length=3667
  warnings.warn(


#PREPARE SIREN DATASET

In [182]:
# Se obtienen las diferentes rutas de los datos, tanto audios como metadata y path para obtener las muestras
file_path = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/UrbanSounds8k/UrbanSound8K/audio'
urbansound8k = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/UrbanSounds8k/UrbanSound8K/metadata/UrbanSound8K.csv')
file_viz = glob.glob('/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/UrbanSounds8k/UrbanSound8K/audio/fold1/*')

In [183]:
#Se organiza dataset para seleccionar la clase de interes y las otras renombrarse como no interes
urbansound8k.loc[urbansound8k["class"] != "siren", "class"] = "non_siren"
#Se cambian los valores de la columna classID para identificar unicamente las dos clases de interes
urbansound8k.loc[urbansound8k["classID"] != 8, "classID"] = 0 #non_event
urbansound8k.loc[urbansound8k["classID"] == 8, "classID"] = 1 #siren

#se cuenta el numero total de muestras de la clase de interes
totalSamplesSiren =  urbansound8k.loc[((urbansound8k["class"]=="siren"))].count()[0]
totalSamplesSiren

#se toma una muestra aleatoria del tamaño de las muestras de la clase de interes para que quede balanceado
dfNonSiren = urbansound8k[urbansound8k['class'] == "non_siren"].sample(n=totalSamplesSiren)
dfSiren = urbansound8k[urbansound8k['class'] == "siren"]

#se unen en un solo dataset las clases de interes con muestras balanceadas
dfCompleteSiren = dfNonSiren.append(dfSiren)
dfCompleteSiren.count()[0]

<ipython-input-183-34a8a78805fd>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfCompleteSiren = dfNonSiren.append(dfSiren)


1858

In [184]:
#Se define funcion para extrar las caracteristicas con la librería librosa, obetiendo los coeficientes ceptrales de frecuencia de Mel
#Se realiza un pading en el tamaño para que concuerden los tamaños de las caracteristicas de entrada al modelo.

def extract_features(file_name, Nmfcc, Nfft, NhopL, NwinL):

    samplerate = 22050
    longitudMaxAudio = 4
    max_pad_len = int(samplerate*longitudMaxAudio/NhopL) + int(samplerate*longitudMaxAudio/NhopL*0.05)  #Calculo longitud de salida de mfcc con 5% de tolerancia para longitud de audios

    try:
      audio, sample_rate = librosa.load(file_name, res_type='soxr_hq')
      mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=Nmfcc, n_fft=Nfft, hop_length=NhopL, win_length=NwinL)
      pad_width = max_pad_len - mfccs.shape[1]
      mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')

    except Exception as e:
      print("Error encountered while parsing file: ", file_name)
      return None
    #print(mfccs.shape)
    return mfccs

In [185]:
#Se realiza la extracción de caracteristicas, teniendo en cuenta la clase, si el sonido es de la carpeta agregada de la clase explosions va y busca este sonido en la carpeta requerida

def get_features(Nmfcc, Nfft, NhopL, NwinL):
  features = []
  # Iterate through each sound file and extract the features
  for index, row in dfCompleteSiren.iterrows():
      if(row["fold"]!=0):
          file_name = os.path.join(os.path.abspath(file_path),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
          #file_name = ""
      elif(row["fold"]==0):
          file_name = os.path.join(os.path.abspath(explosions_path),str(row["slice_file_name"]))
          #file_name = ""
      #print(file_name)
      class_label = row["classID"]
      data = extract_features(file_name, Nmfcc, Nfft, NhopL, NwinL)

      features.append([data, class_label])

  # Convert into a Panda dataframe
  featuresdf = pd.DataFrame(features, columns=['feature','class_label'])
  return featuresdf

In [186]:
def splitFeaturesTrainTest(featuresdf, num_rows, num_columns, num_channels):
  X = np.array(featuresdf.feature.tolist())
  y = np.array(featuresdf.class_label.tolist())

  # Encode the classification labels
  le = LabelEncoder()
  yy = to_categorical(le.fit_transform(y))

  # split the dataset
  from sklearn.model_selection import train_test_split

  x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state = 3)

  x_train = x_train.reshape(x_train.shape[0], num_rows, num_columns, num_channels)
  x_test = x_test.reshape(x_test.shape[0], num_rows, num_columns, num_channels)
  num_labels = yy.shape[1]
  return x_train, x_test, y_train, y_test, num_labels

In [187]:

#1	3	4096	4096	4096	0	2
#2	12	4096	4096	4096	0	3
#3	27	4096	4096	4096	0	3
#4	45	4096	4096	4096	0	5

samplerate = 22050
longitudMaxAudio = 4
Nmfcc = 3
Nfft = 4096
NwinL = 4096
iterableNhopL = 1.0
NhopL =  4096       #int(iterableNhopL*NwinL)
k_size = 2
num_rows = Nmfcc
num_columns = int(samplerate*longitudMaxAudio/NhopL) + int(samplerate*longitudMaxAudio/NhopL*0.05)  #Calculo longitud de salida de mfcc con 5% de tolerancia para longitud de audios
num_channels = 1


featuresdfSiren = get_features(Nmfcc, Nfft, NhopL, NwinL)
x_trainSiren, x_testSiren, y_trainSiren, y_testSiren, num_labelsSiren = splitFeaturesTrainTest(featuresdfSiren, num_rows, num_columns, num_channels)

/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=4096 is too large for input signal of length=3954
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=4096 is too large for input signal of length=1323
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=4096 is too large for input signal of length=3087
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=4096 is too large for input signal of length=3949
  warnings.warn(


In [188]:
x_trainSiren.shape

(1486, 3, 22, 1)

In [189]:
num_labelsSiren

2

In [190]:
num_labelsScream

2

In [191]:
num_labelsGunshot

2

#EVALUATING OG MODELS ARQ 1

In [192]:
modelGunshotTLSave_path = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/gunshot/saved_gunshot_TL_1'
modelGunshotTL = tf.keras.models.load_model(modelGunshotTLSave_path)

modelScreamTLSave_path = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/scream/saved_scream_TL_1'
modelScreamTL = tf.keras.models.load_model(modelScreamTLSave_path)

modelSirenTLSave_path = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/siren/saved_siren_TL_1'
modelSirenTL = tf.keras.models.load_model(modelSirenTLSave_path)
modelSirenTL.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam')


# Evaluating the model on the training and testing set

score = modelScreamTL.evaluate(x_trainScream, y_trainScream, verbose=0)
print("Training Accuracy SCREAM: ", score[1])

score = modelScreamTL.evaluate(x_testScream, y_testScream, verbose=0)
print("Testing Accuracy SCREAM: ", score[1])

y_true = np.argmax(y_testScream,axis=1)
y_pred = np.argmax(modelScreamTL.predict(x_testScream),axis=1)
print('\nConfusion Matrix SCREAM:\n\n')
print(confusion_matrix(y_true,y_pred))
print('\n\nClassification Report SCREAM: \n\n',classification_report(y_true,y_pred))

score = modelGunshotTL.evaluate(x_trainGunshot, y_trainGunshot, verbose=0)
print("Training Accuracy GUN: ", score[1])

score = modelGunshotTL.evaluate(x_testGunshot, y_testGunshot, verbose=0)
print("Testing Accuracy GUN: ", score[1])

y_true = np.argmax(y_testGunshot,axis=1)
y_pred = np.argmax(modelGunshotTL.predict(x_testGunshot),axis=1)
print('\nConfusion Matrix GUN:\n\n')
print(confusion_matrix(y_true,y_pred))
print('\n\nClassification Report GUN: \n\n',classification_report(y_true,y_pred))
print('\n\nClassification Report GUN F1-SCORE: \n\n', float(classification_report(y_true,y_pred).split()[15]))


score = modelSirenTL.evaluate(x_trainSiren, y_trainSiren, verbose=0)
print("Training Accuracy SIREN: ", score[1])

score = modelSirenTL.evaluate(x_testSiren, y_testSiren, verbose=0)
print("Testing Accuracy SIREN: ", score[1])

y_true = np.argmax(y_testSiren,axis=1)
y_pred = np.argmax(modelSirenTL.predict(x_testSiren),axis=1)
print('\nConfusion Matrix SIREN:\n\n')
print(confusion_matrix(y_true,y_pred))
print('\n\nClassification Report SIREN: \n\n',classification_report(y_true,y_pred))
print('\n\nClassification Report SIREN F1-SCORE: \n\n', float(classification_report(y_true,y_pred).split()[15]))

Training Accuracy SCREAM:  0.9936000108718872
Testing Accuracy SCREAM:  0.987261176109314
5/5 [==============================] - 0s 4ms/step

Confusion Matrix SCREAM:


[[83  1]
 [ 1 72]]


Classification Report SCREAM: 

               precision    recall  f1-score   support

           0       0.99      0.99      0.99        84
           1       0.99      0.99      0.99        73

    accuracy                           0.99       157
   macro avg       0.99      0.99      0.99       157
weighted avg       0.99      0.99      0.99       157

Training Accuracy GUN:  0.9515050053596497
Testing Accuracy GUN:  0.95333331823349
5/5 [==============================] - 0s 3ms/step

Confusion Matrix GUN:


[[73  7]
 [ 0 70]]


Classification Report GUN: 

               precision    recall  f1-score   support

           0       1.00      0.91      0.95        80
           1       0.91      1.00      0.95        70

    accuracy                           0.95       150
   macro avg       0.9

#EVALUATING OG MODELS ARQ 4

In [118]:
modelGunshotTLSave_path = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/gunshot/saved_gunshot_TL_4'
modelGunshotTL = tf.keras.models.load_model(modelGunshotTLSave_path)

modelScreamTLSave_path = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/scream/saved_scream_TL_4'
modelScreamTL = tf.keras.models.load_model(modelScreamTLSave_path)

modelSirenTLSave_path = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/siren/saved_siren_TL_4'
modelSirenTL = tf.keras.models.load_model(modelSirenTLSave_path)
modelSirenTL.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam')


# Evaluating the model on the training and testing set

score = modelScreamTL.evaluate(x_trainScream, y_trainScream, verbose=0)
print("Training Accuracy SCREAM: ", score[1])

score = modelScreamTL.evaluate(x_testScream, y_testScream, verbose=0)
print("Testing Accuracy SCREAM: ", score[1])

y_true = np.argmax(y_testScream,axis=1)
y_pred = np.argmax(modelScreamTL.predict(x_testScream),axis=1)
print('\nConfusion Matrix SCREAM:\n\n')
print(confusion_matrix(y_true,y_pred))
print('\n\nClassification Report SCREAM: \n\n',classification_report(y_true,y_pred))

score = modelGunshotTL.evaluate(x_trainGunshot, y_trainGunshot, verbose=0)
print("Training Accuracy GUN: ", score[1])

score = modelGunshotTL.evaluate(x_testGunshot, y_testGunshot, verbose=0)
print("Testing Accuracy GUN: ", score[1])

y_true = np.argmax(y_testGunshot,axis=1)
y_pred = np.argmax(modelGunshotTL.predict(x_testGunshot),axis=1)
print('\nConfusion Matrix GUN:\n\n')
print(confusion_matrix(y_true,y_pred))
print('\n\nClassification Report GUN: \n\n',classification_report(y_true,y_pred))
print('\n\nClassification Report GUN F1-SCORE: \n\n', float(classification_report(y_true,y_pred).split()[15]))


score = modelSirenTL.evaluate(x_trainSiren, y_trainSiren, verbose=0)
print("Training Accuracy SIREN: ", score[1])

score = modelSirenTL.evaluate(x_testSiren, y_testSiren, verbose=0)
print("Testing Accuracy SIREN: ", score[1])

y_true = np.argmax(y_testSiren,axis=1)
y_pred = np.argmax(modelSirenTL.predict(x_testSiren),axis=1)
print('\nConfusion Matrix SIREN:\n\n')
print(confusion_matrix(y_true,y_pred))
print('\n\nClassification Report SIREN: \n\n',classification_report(y_true,y_pred))
print('\n\nClassification Report SIREN F1-SCORE: \n\n', float(classification_report(y_true,y_pred).split()[15]))

Training Accuracy SCREAM:  1.0
Testing Accuracy SCREAM:  0.987261176109314
5/5 [==============================] - 0s 24ms/step

Confusion Matrix SCREAM:


[[83  1]
 [ 1 72]]


Classification Report SCREAM: 

               precision    recall  f1-score   support

           0       0.99      0.99      0.99        84
           1       0.99      0.99      0.99        73

    accuracy                           0.99       157
   macro avg       0.99      0.99      0.99       157
weighted avg       0.99      0.99      0.99       157

Training Accuracy GUN:  0.9765886068344116
Testing Accuracy GUN:  0.9866666793823242
5/5 [==============================] - 0s 21ms/step

Confusion Matrix GUN:


[[79  1]
 [ 1 69]]


Classification Report GUN: 

               precision    recall  f1-score   support

           0       0.99      0.99      0.99        80
           1       0.99      0.99      0.99        70

    accuracy                           0.99       150
   macro avg       0.99      0.99

#Fusion Gun Filters into Scream Filters model

In [193]:
def get_conv_layer(model, layer_index):
    # Find the first convolutional layer starting from the given layer index
    for layer in model.layers[layer_index:]:
        if 'conv' in layer.name:
            return layer
    raise ValueError("No convolutional layer found starting from the given layer index.")

def get_conv_layer(model, layer_index):
    conv_layers = [layer for layer in model.layers if 'conv' in layer.name]

    if layer_index < 0 or layer_index >= len(conv_layers):
        raise ValueError("Invalid layer index.")

    return conv_layers[layer_index]


def change_filters(model1, model2, layer_index):
    layer1 = get_conv_layer(model1, layer_index)
    filters1, bias1 = layer1.get_weights()

    layer2 = get_conv_layer(model2, layer_index)
    filters2, bias2 = layer2.get_weights()
    layer2.set_weights([filters1, bias1])
    layer1.set_weights([filters2, bias2])


In [194]:
def get_conv_layer(model, layer_index):
    conv_layers = [layer for layer in model.layers if 'conv' in layer.name]

    if layer_index < 0 or layer_index >= len(conv_layers):
        raise ValueError("Invalid layer index.")

    return conv_layers[layer_index]



def change_filters(model1, model2, layer_index, transfer_direction):
    layer1 = get_conv_layer(model1, layer_index)
    filters1, bias1 = layer1.get_weights()

    layer2 = get_conv_layer(model2, layer_index)
    filters2, bias2 = layer2.get_weights()

    if transfer_direction == "model1_to_model2":
        layer2.set_weights([filters1, bias1])
    elif transfer_direction == "model2_to_model1":
        layer1.set_weights([filters2, bias2])
    elif transfer_direction == "both_directions":
        layer2.set_weights([filters1, bias1])
        layer1.set_weights([filters2, bias2])
    else:
        raise ValueError("Invalid transfer direction.")

# Ejemplos de uso:
# Cambiar los filtros del modelo 1 al modelo 2
#change_filters(model1, model2, 0, "model1_to_model2")

# Cambiar los filtros del modelo 2 al modelo 1
#change_filters(model1, model2, 0, "model2_to_model1")

# Intercambiar los filtros en ambas direcciones
#change_filters(model1, model2, 0, "both_directions")

#Evaluation model with changes without Criterium

#Evaluation model with changes with GUN and SCREAM MODELS BOTH FROM TL PROGRESSIVE LAYER BY LAYER

#GUN VS SCREAM ARQ 1

In [54]:

modelGunshotTLSave_path_1 = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/gunshot/saved_gunshot_TL_1'
modelGunshotTL_1 = tf.keras.models.load_model(modelGunshotTLSave_path_1)

modelScreamTLSave_path_1 = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/scream/saved_scream_TL_1'
modelScreamTL_1 = tf.keras.models.load_model(modelScreamTLSave_path_1)

modelSirenTLSave_path_1 = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/siren/saved_siren_TL_1'
modelSirenTL_1 = tf.keras.models.load_model(modelSirenTLSave_path_1)


#GUN VS SCREAM
layer_index = 3  # Specify the index of the layer (e.g., 0 for the first convolutional layer)
change_filters(modelGunshotTL_1, modelScreamTL_1, 0, "model1_to_model2")
#change_filters(modelGunshotTL_1, modelScreamTL_1, 0, "model2_to_model1")

#change_filters(modelGunshotTL_1, modelScreamTL_1, 1, "model1_to_model2")
change_filters(modelGunshotTL_1, modelScreamTL_1, 1, "model2_to_model1")

#change_filters(modelGunshotTL_1, modelScreamTL_1, 2, "model1_to_model2")
change_filters(modelGunshotTL_1, modelScreamTL_1, 2, "model2_to_model1")

change_filters(modelGunshotTL_1, modelScreamTL_1, 3, "model1_to_model2")
#change_filters(modelGunshotTL_1, modelScreamTL_1, 3, "model2_to_model1")



# Evaluating the model on the training and testing set
print("GUNSHOT: ")
score = modelGunshotTL_1.evaluate(x_trainGunshot, y_trainGunshot, verbose=0)
print("Training Accuracy: ", score[1])

score = modelGunshotTL_1.evaluate(x_testGunshot, y_testGunshot, verbose=0)
print("Testing Accuracy: ", score[1])

y_true = np.argmax(y_testGunshot,axis=1)
y_pred = np.argmax(modelGunshotTL_1.predict(x_testGunshot),axis=1)
print('\nConfusion Matrix :\n\n')
print(confusion_matrix(y_true,y_pred))
print('\n\nClassification Report : \n\n',classification_report(y_true,y_pred))


print("SCREAM: ")
score = modelScreamTL_1.evaluate(x_trainScream, y_trainScream, verbose=0)
print("Training Accuracy: ", score[1])

score = modelScreamTL_1.evaluate(x_testScream, y_testScream, verbose=0)
print("Testing Accuracy: ", score[1])

y_true = np.argmax(y_testScream,axis=1)
y_pred = np.argmax(modelScreamTL_1.predict(x_testScream),axis=1)
print('\nConfusion Matrix :\n\n')
print(confusion_matrix(y_true,y_pred))
print('\n\nClassification Report : \n\n',classification_report(y_true,y_pred))



#GUN VS SIREN ARQ 1

In [64]:
modelGunshotTLSave_path_1 = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/gunshot/saved_gunshot_TL_1'
modelGunshotTL_1 = tf.keras.models.load_model(modelGunshotTLSave_path_1)

modelScreamTLSave_path_1 = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/scream/saved_scream_TL_1'
modelScreamTL_1 = tf.keras.models.load_model(modelScreamTLSave_path_1)

modelSirenTLSave_path_1 = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/siren/saved_siren_TL_1'
modelSirenTL_1 = tf.keras.models.load_model(modelSirenTLSave_path_1)


#GUN VS SIREN
#layer_index = 3  # Specify the index of the layer (e.g., 0 for the first convolutional layer)

#change_filters(modelGunshotTL_1, modelSirenTL_1, 0, "model1_to_model2")
change_filters(modelGunshotTL_1, modelSirenTL_1, 0, "model2_to_model1")

#change_filters(modelGunshotTL_1, modelSirenTL_1, 1, "model1_to_model2")
change_filters(modelGunshotTL_1, modelSirenTL_1, 1, "model2_to_model1")

#change_filters(modelGunshotTL_1, modelSirenTL_1, 2, "model1_to_model2")
change_filters(modelGunshotTL_1, modelSirenTL_1, 2, "model2_to_model1")

change_filters(modelGunshotTL_1, modelSirenTL_1, 3, "model1_to_model2")
#change_filters(modelGunshotTL_1, modelSirenTL_1, 3, "model2_to_model1")



# Evaluating the model on the training and testing set
print("GUNSHOT: ")
score = modelGunshotTL_1.evaluate(x_trainGunshot, y_trainGunshot, verbose=0)
print("Training Accuracy: ", score[1])

score = modelGunshotTL_1.evaluate(x_testGunshot, y_testGunshot, verbose=0)
print("Testing Accuracy: ", score[1])

y_true = np.argmax(y_testGunshot,axis=1)
y_pred = np.argmax(modelGunshotTL_1.predict(x_testGunshot),axis=1)
print('\nConfusion Matrix :\n\n')
print(confusion_matrix(y_true,y_pred))
print('\n\nClassification Report : \n\n',classification_report(y_true,y_pred))


print("SIREN: ")
score = modelSirenTL_1.evaluate(x_trainSiren, y_trainSiren, verbose=0)
print("Training Accuracy: ", score[1])

score = modelSirenTL_1.evaluate(x_testSiren, y_testSiren, verbose=0)
print("Testing Accuracy: ", score[1])

y_true = np.argmax(y_testSiren,axis=1)
y_pred = np.argmax(modelSirenTL_1.predict(x_testSiren),axis=1)
print('\nConfusion Matrix :\n\n')
print(confusion_matrix(y_true,y_pred))
print('\n\nClassification Report : \n\n',classification_report(y_true,y_pred))


GUNSHOT: 
Training Accuracy:  0.7006688714027405
Testing Accuracy:  0.6733333468437195
5/5 [==============================] - 0s 4ms/step

Confusion Matrix :


[[37 43]
 [ 6 64]]


Classification Report : 

               precision    recall  f1-score   support

           0       0.86      0.46      0.60        80
           1       0.60      0.91      0.72        70

    accuracy                           0.67       150
   macro avg       0.73      0.69      0.66       150
weighted avg       0.74      0.67      0.66       150

SIREN: 
Training Accuracy:  0.8270524740219116
Testing Accuracy:  0.8172042965888977
12/12 [==============================] - 0s 4ms/step

Confusion Matrix :


[[154  38]
 [ 30 150]]


Classification Report : 

               precision    recall  f1-score   support

           0       0.84      0.80      0.82       192
           1       0.80      0.83      0.82       180

    accuracy                           0.82       372
   macro avg       0.82      0.82  

#SCREAM VS SIREN ARQ 1

In [72]:
modelGunshotTLSave_path_1 = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/gunshot/saved_gunshot_TL_1'
modelGunshotTL_1 = tf.keras.models.load_model(modelGunshotTLSave_path_1)

modelScreamTLSave_path_1 = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/scream/saved_scream_TL_1'
modelScreamTL_1 = tf.keras.models.load_model(modelScreamTLSave_path_1)

modelSirenTLSave_path_1 = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/siren/saved_siren_TL_1'
modelSirenTL_1 = tf.keras.models.load_model(modelSirenTLSave_path_1)


#SCREAM VS SIREN
#layer_index = 3  # Specify the index of the layer (e.g., 0 for the first convolutional layer)

#change_filters(modelScreamTL_1, modelSirenTL_1, 0, "model1_to_model2")
change_filters(modelScreamTL_1, modelSirenTL_1, 0, "model2_to_model1")

#change_filters(modelScreamTL_1, modelSirenTL_1, 1, "model1_to_model2")
change_filters(modelScreamTL_1, modelSirenTL_1, 1, "model2_to_model1")

#change_filters(modelScreamTL_1, modelSirenTL_1, 2, "model1_to_model2")
change_filters(modelScreamTL_1, modelSirenTL_1, 2, "model2_to_model1")

#change_filters(modelScreamTL_1, modelSirenTL_1, 3, "model1_to_model2")
change_filters(modelScreamTL_1, modelSirenTL_1, 3, "model2_to_model1")



# Evaluating the model on the training and testing set
print("SCREAM: ")
score = modelScreamTL_1.evaluate(x_trainScream, y_trainScream, verbose=0)
print("Training Accuracy: ", score[1])

score = modelScreamTL_1.evaluate(x_testScream, y_testScream, verbose=0)
print("Testing Accuracy: ", score[1])

y_true = np.argmax(y_testScream,axis=1)
y_pred = np.argmax(modelScreamTL_1.predict(x_testScream),axis=1)
print('\nConfusion Matrix :\n\n')
print(confusion_matrix(y_true,y_pred))
print('\n\nClassification Report : \n\n',classification_report(y_true,y_pred))


print("SIREN: ")
score = modelSirenTL_1.evaluate(x_trainSiren, y_trainSiren, verbose=0)
print("Training Accuracy: ", score[1])

score = modelSirenTL_1.evaluate(x_testSiren, y_testSiren, verbose=0)
print("Testing Accuracy: ", score[1])

y_true = np.argmax(y_testSiren,axis=1)
y_pred = np.argmax(modelSirenTL_1.predict(x_testSiren),axis=1)
print('\nConfusion Matrix :\n\n')
print(confusion_matrix(y_true,y_pred))
print('\n\nClassification Report : \n\n',classification_report(y_true,y_pred))


SCREAM: 
Training Accuracy:  0.795199990272522
Testing Accuracy:  0.808917224407196
5/5 [==============================] - 0s 6ms/step

Confusion Matrix :


[[79  5]
 [25 48]]


Classification Report : 

               precision    recall  f1-score   support

           0       0.76      0.94      0.84        84
           1       0.91      0.66      0.76        73

    accuracy                           0.81       157
   macro avg       0.83      0.80      0.80       157
weighted avg       0.83      0.81      0.80       157

SIREN: 
Training Accuracy:  0.8270524740219116
Testing Accuracy:  0.8172042965888977
12/12 [==============================] - 0s 4ms/step

Confusion Matrix :


[[154  38]
 [ 30 150]]


Classification Report : 

               precision    recall  f1-score   support

           0       0.84      0.80      0.82       192
           1       0.80      0.83      0.82       180

    accuracy                           0.82       372
   macro avg       0.82      0.82     

#GUN VS SCREAM ARQ 4

In [131]:

modelGunshotTLSave_path_4 = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/gunshot/saved_gunshot_TL_4'
modelGunshotTL_4 = tf.keras.models.load_model(modelGunshotTLSave_path_4)

modelScreamTLSave_path_4 = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/scream/saved_scream_TL_4'
modelScreamTL_4 = tf.keras.models.load_model(modelScreamTLSave_path_4)

modelSirenTLSave_path_4 = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/siren/saved_siren_TL_4'
modelSirenTL_4 = tf.keras.models.load_model(modelSirenTLSave_path_4)


#GUN VS SCREAM
layer_index = 3  # Specify the index of the layer (e.g., 0 for the first convolutional layer)

change_filters(modelGunshotTL_4, modelScreamTL_4, 0, "model1_to_model2")
#change_filters(modelGunshotTL_4, modelScreamTL_4, 0, "model2_to_model1")

change_filters(modelGunshotTL_4, modelScreamTL_4, 1, "model1_to_model2")
#change_filters(modelGunshotTL_4, modelScreamTL_4, 1, "model2_to_model1")

change_filters(modelGunshotTL_4, modelScreamTL_4, 2, "model1_to_model2")
#change_filters(modelGunshotTL_4, modelScreamTL_4, 2, "model2_to_model1")

#change_filters(modelGunshotTL_4, modelScreamTL_4, 3, "model1_to_model2")
change_filters(modelGunshotTL_4, modelScreamTL_4, 3, "model2_to_model1")



# Evaluating the model on the training and testing set
print("GUNSHOT: ")
score = modelGunshotTL_4.evaluate(x_trainGunshot, y_trainGunshot, verbose=0)
print("Training Accuracy: ", score[1])

score = modelGunshotTL_4.evaluate(x_testGunshot, y_testGunshot, verbose=0)
print("Testing Accuracy: ", score[1])

y_true = np.argmax(y_testGunshot,axis=1)
y_pred = np.argmax(modelGunshotTL_4.predict(x_testGunshot),axis=1)
print('\nConfusion Matrix :\n\n')
print(confusion_matrix(y_true,y_pred))
print('\n\nClassification Report : \n\n',classification_report(y_true,y_pred))


print("SCREAM: ")
score = modelScreamTL_4.evaluate(x_trainScream, y_trainScream, verbose=0)
print("Training Accuracy: ", score[1])

score = modelScreamTL_4.evaluate(x_testScream, y_testScream, verbose=0)
print("Testing Accuracy: ", score[1])

y_true = np.argmax(y_testScream,axis=1)
y_pred = np.argmax(modelScreamTL_4.predict(x_testScream),axis=1)
print('\nConfusion Matrix :\n\n')
print(confusion_matrix(y_true,y_pred))
print('\n\nClassification Report : \n\n',classification_report(y_true,y_pred))



GUNSHOT: 
Training Accuracy:  0.8127090334892273
Testing Accuracy:  0.8133333325386047
5/5 [==============================] - 0s 22ms/step

Confusion Matrix :


[[55 25]
 [ 3 67]]


Classification Report : 

               precision    recall  f1-score   support

           0       0.95      0.69      0.80        80
           1       0.73      0.96      0.83        70

    accuracy                           0.81       150
   macro avg       0.84      0.82      0.81       150
weighted avg       0.85      0.81      0.81       150

SCREAM: 
Training Accuracy:  0.9552000164985657
Testing Accuracy:  0.9617834687232971
5/5 [==============================] - 0s 23ms/step

Confusion Matrix :


[[80  4]
 [ 2 71]]


Classification Report : 

               precision    recall  f1-score   support

           0       0.98      0.95      0.96        84
           1       0.95      0.97      0.96        73

    accuracy                           0.96       157
   macro avg       0.96      0.96     

#GUN VS SIREN ARQ 4

In [139]:

modelGunshotTLSave_path_4 = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/gunshot/saved_gunshot_TL_4'
modelGunshotTL_4 = tf.keras.models.load_model(modelGunshotTLSave_path_4)

modelScreamTLSave_path_4 = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/scream/saved_scream_TL_4'
modelScreamTL_4 = tf.keras.models.load_model(modelScreamTLSave_path_4)

modelSirenTLSave_path_4 = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/siren/saved_siren_TL_4'
modelSirenTL_4 = tf.keras.models.load_model(modelSirenTLSave_path_4)


#GUN VS SIREN
#layer_index = 3  # Specify the index of the layer (e.g., 0 for the first convolutional layer)

#change_filters(modelGunshotTL_4, modelSirenTL_4, 0, "model1_to_model2")
change_filters(modelGunshotTL_4, modelSirenTL_4, 0, "model2_to_model1")

#change_filters(modelGunshotTL_4, modelSirenTL_4, 1, "model1_to_model2")
change_filters(modelGunshotTL_4, modelSirenTL_4, 1, "model2_to_model1")

change_filters(modelGunshotTL_4, modelSirenTL_4, 2, "model1_to_model2")
#change_filters(modelGunshotTL_4, modelSirenTL_4, 2, "model2_to_model1")

#change_filters(modelGunshotTL_4, modelSirenTL_4, 3, "model1_to_model2")
change_filters(modelGunshotTL_4, modelSirenTL_4, 3, "model2_to_model1")



# Evaluating the model on the training and testing set
print("GUNSHOT: ")
score = modelGunshotTL_4.evaluate(x_trainGunshot, y_trainGunshot, verbose=0)
print("Training Accuracy: ", score[1])

score = modelGunshotTL_4.evaluate(x_testGunshot, y_testGunshot, verbose=0)
print("Testing Accuracy: ", score[1])

y_true = np.argmax(y_testGunshot,axis=1)
y_pred = np.argmax(modelGunshotTL_4.predict(x_testGunshot),axis=1)
print('\nConfusion Matrix :\n\n')
print(confusion_matrix(y_true,y_pred))
print('\n\nClassification Report : \n\n',classification_report(y_true,y_pred))


print("SIREN: ")
score = modelSirenTL_4.evaluate(x_trainSiren, y_trainSiren, verbose=0)
print("Training Accuracy: ", score[1])

score = modelSirenTL_4.evaluate(x_testSiren, y_testSiren, verbose=0)
print("Testing Accuracy: ", score[1])

y_true = np.argmax(y_testSiren,axis=1)
y_pred = np.argmax(modelSirenTL_4.predict(x_testSiren),axis=1)
print('\nConfusion Matrix :\n\n')
print(confusion_matrix(y_true,y_pred))
print('\n\nClassification Report : \n\n',classification_report(y_true,y_pred))

GUNSHOT: 
Training Accuracy:  0.8277592062950134
Testing Accuracy:  0.8266666531562805
5/5 [==============================] - 1s 61ms/step

Confusion Matrix :


[[68 12]
 [14 56]]


Classification Report : 

               precision    recall  f1-score   support

           0       0.83      0.85      0.84        80
           1       0.82      0.80      0.81        70

    accuracy                           0.83       150
   macro avg       0.83      0.82      0.83       150
weighted avg       0.83      0.83      0.83       150

SIREN: 
Training Accuracy:  0.9784656763076782
Testing Accuracy:  0.9596773982048035
12/12 [==============================] - 1s 33ms/step

Confusion Matrix :


[[188   4]
 [ 11 169]]


Classification Report : 

               precision    recall  f1-score   support

           0       0.94      0.98      0.96       192
           1       0.98      0.94      0.96       180

    accuracy                           0.96       372
   macro avg       0.96      0.96

#SCREAM VS SIREN ARQ 4

In [148]:

modelGunshotTLSave_path_4 = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/gunshot/saved_gunshot_TL_4'
modelGunshotTL_4 = tf.keras.models.load_model(modelGunshotTLSave_path_4)

modelScreamTLSave_path_4 = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/scream/saved_scream_TL_4'
modelScreamTL_4 = tf.keras.models.load_model(modelScreamTLSave_path_4)

modelSirenTLSave_path_4 = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/siren/saved_siren_TL_4'
modelSirenTL_4 = tf.keras.models.load_model(modelSirenTLSave_path_4)



#SCREAM VS SIREN
#layer_index = 3  # Specify the index of the layer (e.g., 0 for the first convolutional layer)

#change_filters(modelScreamTL_4, modelSirenTL_4, 0, "model1_to_model2")
change_filters(modelScreamTL_4, modelSirenTL_4, 0, "model2_to_model1")

#change_filters(modelScreamTL_4, modelSirenTL_4, 1, "model1_to_model2")
change_filters(modelScreamTL_4, modelSirenTL_4, 1, "model2_to_model1")

#change_filters(modelScreamTL_4, modelSirenTL_4, 2, "model1_to_model2")
change_filters(modelScreamTL_4, modelSirenTL_4, 2, "model2_to_model1")

change_filters(modelScreamTL_4, modelSirenTL_4, 3, "model1_to_model2")
#change_filters(modelScreamTL_4, modelSirenTL_4, 3, "model2_to_model1")



# Evaluating the model on the training and testing set
print("SCREAM: ")
score = modelScreamTL_4.evaluate(x_trainScream, y_trainScream, verbose=0)
print("Training Accuracy: ", score[1])

score = modelScreamTL_4.evaluate(x_testScream, y_testScream, verbose=0)
print("Testing Accuracy: ", score[1])

y_true = np.argmax(y_testScream,axis=1)
y_pred = np.argmax(modelScreamTL_4.predict(x_testScream),axis=1)
print('\nConfusion Matrix :\n\n')
print(confusion_matrix(y_true,y_pred))
print('\n\nClassification Report : \n\n',classification_report(y_true,y_pred))


print("SIREN: ")
score = modelSirenTL_4.evaluate(x_trainSiren, y_trainSiren, verbose=0)
print("Training Accuracy: ", score[1])

score = modelSirenTL_4.evaluate(x_testSiren, y_testSiren, verbose=0)
print("Testing Accuracy: ", score[1])

y_true = np.argmax(y_testSiren,axis=1)
y_pred = np.argmax(modelSirenTL_4.predict(x_testSiren),axis=1)
print('\nConfusion Matrix :\n\n')
print(confusion_matrix(y_true,y_pred))
print('\n\nClassification Report : \n\n',classification_report(y_true,y_pred))

SCREAM: 
Training Accuracy:  0.9872000217437744
Testing Accuracy:  0.9808917045593262
5/5 [==============================] - 1s 70ms/step

Confusion Matrix :


[[83  1]
 [ 2 71]]


Classification Report : 

               precision    recall  f1-score   support

           0       0.98      0.99      0.98        84
           1       0.99      0.97      0.98        73

    accuracy                           0.98       157
   macro avg       0.98      0.98      0.98       157
weighted avg       0.98      0.98      0.98       157

SIREN: 
Training Accuracy:  0.824360728263855
Testing Accuracy:  0.8198924660682678
12/12 [==============================] - 0s 22ms/step

Confusion Matrix :


[[129  63]
 [  4 176]]


Classification Report : 

               precision    recall  f1-score   support

           0       0.97      0.67      0.79       192
           1       0.74      0.98      0.84       180

    accuracy                           0.82       372
   macro avg       0.85      0.82  

#Creando Nuevo Modelo FUSIONADO

In [195]:
# Constructing FUSED model:
def getModel(num_rows, num_columns, num_channels, num_labels, k_size):
    model = Sequential()
    model.add(Conv2D(filters=16, kernel_size=k_size, input_shape=(num_rows, num_columns, num_channels), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
    model.add(Dropout(0.2))

    model.add(Conv2D(filters=32, kernel_size=k_size, activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
    model.add(Dropout(0.2))

    model.add(Conv2D(filters=64, kernel_size=k_size, activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
    model.add(Dropout(0.2))

    model.add(Conv2D(filters=128, kernel_size=k_size, activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
    model.add(Dropout(0.2))
    model.add(GlobalAveragePooling2D())
    model.add(Flatten())
    model.add(Dense(num_labels, activation='softmax'))
    return model

In [196]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, GlobalAveragePooling2D, Flatten, Dense
from tensorflow.keras.models import Model

def getModelFunctional(num_rows, num_columns, num_channels, num_labels, k_size):
    inputs = Input(shape=(num_rows, num_columns, num_channels))
    x = Conv2D(filters=16, kernel_size=k_size, activation='relu', padding='same')(inputs)
    x = MaxPooling2D(pool_size=(2,2), padding='same')(x)
    x = Dropout(0.2)(x)

    x = Conv2D(filters=32, kernel_size=k_size, activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2,2), padding='same')(x)
    x = Dropout(0.2)(x)

    x = Conv2D(filters=64, kernel_size=k_size, activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2,2), padding='same')(x)
    x = Dropout(0.2)(x)

    x = Conv2D(filters=128, kernel_size=k_size, activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2,2), padding='same')(x)
    x = Dropout(0.2)(x)

    x = GlobalAveragePooling2D()(x)
    x = Flatten()(x)
    outputs = Dense(num_labels, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model

In [197]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, GlobalAveragePooling2D, Flatten, Dense
from tensorflow.keras.models import Model

def getModelFunctionalModified(num_rows, num_columns, num_channels, num_labels, k_size):
    inputs = Input(shape=(num_rows, num_columns, num_channels))
    x = Conv2D(filters=16, kernel_size=k_size, activation='relu', padding='same')(inputs)
    x = MaxPooling2D(pool_size=(2,2), padding='same')(x)
    x = Dropout(0.2)(x)

    x = Conv2D(filters=32, kernel_size=k_size, activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2,2), padding='same')(x)
    x = Dropout(0.2)(x)

    x = Conv2D(filters=64, kernel_size=k_size, activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2,2), padding='same')(x)
    x = Dropout(0.2)(x)

    x = Conv2D(filters=128, kernel_size=k_size, activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2,2), padding='same')(x)
    x = Dropout(0.2)(x)

    x = GlobalAveragePooling2D()(x)
    x = Flatten()(x)

    output1 = Dense(num_labels, activation='softmax', name='output1')(x)
    output2 = Dense(num_labels, activation='softmax', name='output2')(x)

    model = Model(inputs=inputs, outputs=[output1, output2])
    return model

In [198]:
def get_layer_with_params(model, layer_index):
    layers_with_params = [layer for layer in model.layers if len(layer.trainable_weights) > 0]

    if layer_index < 0 or layer_index >= len(layers_with_params):
        raise ValueError("Invalid layer index.")

    return layers_with_params[layer_index]

def transfer_layer_weights(layer_from, layer_to):
    if len(layer_from.trainable_weights) == 0 or len(layer_to.trainable_weights) == 0 or ((len(layer_from.trainable_weights)) != (len(layer_to.trainable_weights))):
        return False  # Ambas capas deben tener parámetros entrenables

    weights_from, biases_from = layer_from.get_weights()
    layer_to.set_weights([weights_from, biases_from])
    return True

def transfer_parameters_between_models(model_from, layer_index_from, model_to, layer_index_to):
    layer_from = get_layer_with_params(model_from, layer_index_from)
    layer_to = get_layer_with_params(model_to, layer_index_to)

    if transfer_layer_weights(layer_from, layer_to):
        print(f"Transferred weights from layer '{layer_from.name}' to layer '{layer_to.name}'.")
    else:
        print(f"Layers '{layer_from.name}' and '{layer_to.name}' don't have trainable parameters.")

#EVALUAR FUSIÓN CON CRITERIO BASADO EN MÁXIMO VALOR DE PARÁMETRO


In [199]:
def transfer_max_abs_conv_weights(model_from_1, model_from_2, model_to):
    layer_names = [layer.name for layer in model_from_1.layers if 'conv' in layer.name]

    for layer_index in range(len(layer_names)):
      #print(layer_index)
      layer_from_1 = get_layer_with_params(model_from_1, layer_index)
      layer_from_2 = get_layer_with_params(model_from_2, layer_index)
      layer_to = get_layer_with_params(model_to, layer_index)


      weights_from_1, biases_from_1 = layer_from_1.get_weights()
      weights_from_2, biases_from_2 = layer_from_2.get_weights()

      #print(weights_from_1.shape)
      #print(weights_from_2.shape)

      max_abs_weights = np.array([np.maximum(np.abs(w1), np.abs(w2)) for w1, w2 in zip(weights_from_1, weights_from_2)])
      max_abs_biases = np.array([np.maximum(np.abs(b1), np.abs(b2)) for b1, b2 in zip(biases_from_1, biases_from_2)])

      #print(max_abs_weights.shape)

      layer_to.set_weights([max_abs_weights, max_abs_biases])

    return model_to

#EVALUAR FUSIÓN CON CRITERIO BASADO EN MÍNIMO VALOR DE PARÁMETRO

In [200]:
def transfer_min_abs_conv_weights(model_from_1, model_from_2, model_to):
    layer_names = [layer.name for layer in model_from_1.layers if 'conv' in layer.name]

    for layer_index in range(len(layer_names)):
        #print(layer_index)
        layer_from_1 = get_layer_with_params(model_from_1, layer_index)
        layer_from_2 = get_layer_with_params(model_from_2, layer_index)
        layer_to = get_layer_with_params(model_to, layer_index)

        weights_from_1, biases_from_1 = layer_from_1.get_weights()
        weights_from_2, biases_from_2 = layer_from_2.get_weights()

        #print(weights_from_1.shape)
        #print(weights_from_2.shape)

        min_abs_weights = np.array([np.minimum(np.abs(w1), np.abs(w2)) for w1, w2 in zip(weights_from_1, weights_from_2)])
        min_abs_biases = np.array([np.minimum(np.abs(b1), np.abs(b2)) for b1, b2 in zip(biases_from_1, biases_from_2)])

        #print(min_abs_weights.shape)

        layer_to.set_weights([min_abs_weights, min_abs_biases])

    return model_to

#EVALUAR FUSIÓN CON CRITERIO BASADO EN VALOR PROMEDIO DE PARÁMETROS

In [201]:
def transfer_avg_conv_weights(model_from_1, model_from_2, model_to):
    layer_names = [layer.name for layer in model_from_1.layers if 'conv' in layer.name]

    for layer_index in range(len(layer_names)):
        #print(layer_index)
        layer_from_1 = get_layer_with_params(model_from_1, layer_index)
        layer_from_2 = get_layer_with_params(model_from_2, layer_index)
        layer_to = get_layer_with_params(model_to, layer_index)

        weights_from_1, biases_from_1 = layer_from_1.get_weights()
        weights_from_2, biases_from_2 = layer_from_2.get_weights()

        #print(weights_from_1.shape)
        #print(weights_from_2.shape)

        avg_weights = np.array([(w1 + w2) / 2 for w1, w2 in zip(weights_from_1, weights_from_2)])
        avg_biases = np.array([(b1 + b2) / 2 for b1, b2 in zip(biases_from_1, biases_from_2)])

        #print(avg_weights.shape)

        layer_to.set_weights([avg_weights, avg_biases])

    return model_to

#GUN VS SCREAM ARQ 1 MAX/MIN/AVG VALUE IN LAYER

In [204]:
#1	3	4096	4096	4096	0	2
#2	12	4096	4096	4096	0	3
#3	27	4096	4096	4096	0	3
#4	45	4096	4096	4096	0	5

samplerate = 22050
longitudMaxAudio = 4
Nmfcc = 3
Nfft = 4096
NwinL = 4096
iterableNhopL = 1.0
NhopL =  4096       #int(iterableNhopL*NwinL)
k_size = 2
num_rows = Nmfcc
num_columns = int(samplerate*longitudMaxAudio/NhopL) + int(samplerate*longitudMaxAudio/NhopL*0.05)  #Calculo longitud de salida de mfcc con 5% de tolerancia para longitud de audios
num_channels = 1

num_labels = 2 #4

#model = getModel(num_rows, num_columns, num_channels, num_labels, k_size)
#model = getModelFunctional(num_rows, num_columns, num_channels, num_labels, k_size)
Fusedmodel = getModelFunctionalModified(num_rows, num_columns, num_channels, num_labels, k_size)

# Compila el modelo (es necesario antes de la evaluación)
Fusedmodel.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#TRAINED MODELS TO FUSE
modelGunshotTLSave_path_1 = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/gunshot/saved_gunshot_TL_1'
modelGunshotTL_1 = tf.keras.models.load_model(modelGunshotTLSave_path_1)

modelScreamTLSave_path_1 = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/scream/saved_scream_TL_1'
modelScreamTL_1 = tf.keras.models.load_model(modelScreamTLSave_path_1)

#layer_index = 3  # Specify the index of the layer (e.g., 0 for the first convolutional layer)
#change_filters(modelGunshotTL, modelScreamTL, 0, "model1_to_model2")
#change_filters(modelGunshotTL, modelScreamTL, 1, "model1_to_model2")
#change_filters(modelGunshotTL, modelScreamTL, 2, "model2_to_model1")
#change_filters(modelGunshotTL, modelScreamTL, layer_index, "model1_to_model2")

# Iterar sobre las capas y verificar la cantidad de parámetros entrenables
for layer in Fusedmodel.layers:
    trainable_params_count = layer.count_params()
    if trainable_params_count > 0:
        print("Layer Name:", layer.name)
        print("Trainable Parameters Count:", trainable_params_count)
print("------------------------")

# Iterar sobre las capas y verificar la cantidad de parámetros entrenables
for layer in modelGunshotTL_1.layers:
    trainable_params_count = layer.count_params()
    if trainable_params_count > 0:
        print("Layer Name:", layer.name)
        print("Trainable Parameters Count:", trainable_params_count)
print("------------------------")


#transfer_parameters_between_models(modelGunshotTL, 0, Fusedmodel, 0)
#transfer_parameters_between_models(modelGunshotTL, 1, Fusedmodel, 1)
#transfer_parameters_between_models(modelScreamTL, 2, Fusedmodel, 2)
#transfer_parameters_between_models(modelGunshotTL, 3, Fusedmodel, 3)



#transfer_max_abs_conv_weights(modelGunshotTL_1, modelScreamTL_1, Fusedmodel)
#transfer_min_abs_conv_weights(modelGunshotTL_1, modelScreamTL_1, Fusedmodel)
transfer_avg_conv_weights(modelGunshotTL_1, modelScreamTL_1, Fusedmodel)

transfer_parameters_between_models(modelGunshotTL_1, 4, Fusedmodel, 4)
transfer_parameters_between_models(modelScreamTL_1, 4, Fusedmodel, 5)


#Fusedmodel.summary()


# Evaluating the FUSED model on the training and testing sets

scoreGunshot = Fusedmodel.evaluate(x_trainGunshot, [y_trainGunshot, np.resize(y_trainScream, (y_trainGunshot.shape[0], *y_trainScream.shape[1:]))], verbose=0)
print("OUT Gunshot: ", scoreGunshot)
print("Training Accuracy for Gunshot: ", scoreGunshot[3])

scoreGunshot = Fusedmodel.evaluate(x_testGunshot, [y_testGunshot, np.resize(y_testScream, (y_testGunshot.shape[0], *y_testScream.shape[1:]))], verbose=0)
print("OUT Gunshot: ", scoreGunshot)
print("Testing Accuracy for Gunshot: ", scoreGunshot[3])



scoreScream= Fusedmodel.evaluate(x_trainScream, [np.resize(y_trainGunshot, (y_trainScream.shape[0], *y_trainGunshot.shape[1:])), y_trainScream], verbose=0)
print("OUT Scream: ", scoreScream)
print("Training Accuracy for Scream: ", scoreScream[4])

scoreScream= Fusedmodel.evaluate(x_testScream, [np.resize(y_testGunshot, (y_testScream.shape[0], *y_testGunshot.shape[1:])), y_testScream], verbose=0)
print("OUT Scream: ", scoreScream)
print("Testing Accuracy for Scream: ", scoreScream[4])


y_true = np.argmax(y_testGunshot,axis=1)
y_pred = np.argmax(Fusedmodel.predict(x_testGunshot)[0],axis=1)
print('\nConfusion Matrix GUNSHOT:\n\n')
print('SHAPEEE : ',y_true.shape)
print('SHAPEEE : ',y_pred.shape)
print(confusion_matrix(y_true,y_pred))
print('\n\nClassification Report : \n\n',classification_report(y_true,y_pred))

y_true = np.argmax(y_testScream,axis=1)
y_pred = np.argmax(Fusedmodel.predict(x_testScream)[1],axis=1)
print('\nConfusion Matrix SCREAM:\n\n')
print('SHAPEEE : ',y_true.shape)
print('SHAPEEE : ',y_pred.shape)
print(confusion_matrix(y_true,y_pred))
print('\n\nClassification Report : \n\n',classification_report(y_true,y_pred))

Layer Name: conv2d_88
Trainable Parameters Count: 80
Layer Name: conv2d_89
Trainable Parameters Count: 2080
Layer Name: conv2d_90
Trainable Parameters Count: 8256
Layer Name: conv2d_91
Trainable Parameters Count: 32896
Layer Name: output1
Trainable Parameters Count: 258
Layer Name: output2
Trainable Parameters Count: 258
------------------------
Layer Name: conv2d
Trainable Parameters Count: 80
Layer Name: conv2d_1
Trainable Parameters Count: 2080
Layer Name: conv2d_2
Trainable Parameters Count: 8256
Layer Name: conv2d_3
Trainable Parameters Count: 32896
Layer Name: dense
Trainable Parameters Count: 258
------------------------
Transferred weights from layer 'dense' to layer 'output1'.
Transferred weights from layer 'dense' to layer 'output2'.
OUT Gunshot:  [1.90944242477417, 0.2235431969165802, 1.685899257659912, 0.9180601835250854, 0.5100334286689758]
Training Accuracy for Gunshot:  0.9180601835250854
OUT Gunshot:  [1.7039003372192383, 0.23468945920467377, 1.4692109823226929, 0.91333

#GUN VS SIREN ARQ 1 MAX/MIN/AVG VALUE IN LAYER

In [207]:
#1	3	4096	4096	4096	0	2
#2	12	4096	4096	4096	0	3
#3	27	4096	4096	4096	0	3
#4	45	4096	4096	4096	0	5

samplerate = 22050
longitudMaxAudio = 4
Nmfcc = 3
Nfft = 4096
NwinL = 4096
iterableNhopL = 1.0
NhopL =  4096       #int(iterableNhopL*NwinL)
k_size = 2
num_rows = Nmfcc
num_columns = int(samplerate*longitudMaxAudio/NhopL) + int(samplerate*longitudMaxAudio/NhopL*0.05)  #Calculo longitud de salida de mfcc con 5% de tolerancia para longitud de audios
num_channels = 1

num_labels = 2 #4

#model = getModel(num_rows, num_columns, num_channels, num_labels, k_size)
#model = getModelFunctional(num_rows, num_columns, num_channels, num_labels, k_size)
Fusedmodel = getModelFunctionalModified(num_rows, num_columns, num_channels, num_labels, k_size)

# Compila el modelo (es necesario antes de la evaluación)
Fusedmodel.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#TRAINED MODELS TO FUSE
modelGunshotTLSave_path_1 = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/gunshot/saved_gunshot_TL_1'
modelGunshotTL_1 = tf.keras.models.load_model(modelGunshotTLSave_path_1)

modelSirenTLSave_path_1 = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/siren/saved_siren_TL_1'
modelSirenTL_1 = tf.keras.models.load_model(modelSirenTLSave_path_1)

#layer_index = 3  # Specify the index of the layer (e.g., 0 for the first convolutional layer)
#change_filters(modelGunshotTL, modelScreamTL, 0, "model1_to_model2")
#change_filters(modelGunshotTL, modelScreamTL, 1, "model1_to_model2")
#change_filters(modelGunshotTL, modelScreamTL, 2, "model2_to_model1")
#change_filters(modelGunshotTL, modelScreamTL, layer_index, "model1_to_model2")

# Iterar sobre las capas y verificar la cantidad de parámetros entrenables
for layer in Fusedmodel.layers:
    trainable_params_count = layer.count_params()
    if trainable_params_count > 0:
        print("Layer Name:", layer.name)
        print("Trainable Parameters Count:", trainable_params_count)
print("------------------------")

# Iterar sobre las capas y verificar la cantidad de parámetros entrenables
for layer in modelGunshotTL_1.layers:
    trainable_params_count = layer.count_params()
    if trainable_params_count > 0:
        print("Layer Name:", layer.name)
        print("Trainable Parameters Count:", trainable_params_count)
print("------------------------")


#transfer_parameters_between_models(modelGunshotTL, 0, Fusedmodel, 0)
#transfer_parameters_between_models(modelGunshotTL, 1, Fusedmodel, 1)
#transfer_parameters_between_models(modelScreamTL, 2, Fusedmodel, 2)
#transfer_parameters_between_models(modelGunshotTL, 3, Fusedmodel, 3)



#transfer_max_abs_conv_weights(modelGunshotTL_1, modelSirenTL_1, Fusedmodel)
#transfer_min_abs_conv_weights(modelGunshotTL_1, modelSirenTL_1, Fusedmodel)
transfer_avg_conv_weights(modelGunshotTL_1, modelSirenTL_1, Fusedmodel)

transfer_parameters_between_models(modelGunshotTL_1, 4, Fusedmodel, 4)
transfer_parameters_between_models(modelSirenTL_1, 4, Fusedmodel, 5)


#Fusedmodel.summary()


# Evaluating the FUSED model on the training and testing sets

scoreGunshot = Fusedmodel.evaluate(x_trainGunshot, [y_trainGunshot, np.resize(y_trainSiren, (y_trainGunshot.shape[0], *y_trainSiren.shape[1:]))], verbose=0)
print("OUT Gunshot: ", scoreGunshot)
print("Training Accuracy for Gunshot: ", scoreGunshot[3])

scoreGunshot = Fusedmodel.evaluate(x_testGunshot, [y_testGunshot, np.resize(y_testSiren, (y_testGunshot.shape[0], *y_testSiren.shape[1:]))], verbose=0)
print("OUT Gunshot: ", scoreGunshot)
print("Testing Accuracy for Gunshot: ", scoreGunshot[3])



scoreSiren= Fusedmodel.evaluate(x_trainSiren, [np.resize(y_trainGunshot, (y_trainSiren.shape[0], *y_trainGunshot.shape[1:])), y_trainSiren], verbose=0)
print("OUT Siren: ", scoreSiren)
print("Training Accuracy for Siren: ", scoreSiren[4])

scoreSiren= Fusedmodel.evaluate(x_testSiren, [np.resize(y_testGunshot, (y_testSiren.shape[0], *y_testGunshot.shape[1:])), y_testSiren], verbose=0)
print("OUT Siren: ", scoreSiren)
print("Testing Accuracy for Siren: ", scoreSiren[4])


y_true = np.argmax(y_testGunshot,axis=1)
y_pred = np.argmax(Fusedmodel.predict(x_testGunshot)[0],axis=1)
print('\nConfusion Matrix GUNSHOT:\n\n')
print('SHAPEEE : ',y_true.shape)
print('SHAPEEE : ',y_pred.shape)
print(confusion_matrix(y_true,y_pred))
print('\n\nClassification Report : \n\n',classification_report(y_true,y_pred))

y_true = np.argmax(y_testSiren,axis=1)
y_pred = np.argmax(Fusedmodel.predict(x_testSiren)[1],axis=1)
print('\nConfusion Matrix SIREN:\n\n')
print('SHAPEEE : ',y_true.shape)
print('SHAPEEE : ',y_pred.shape)
print(confusion_matrix(y_true,y_pred))
print('\n\nClassification Report : \n\n',classification_report(y_true,y_pred))

Layer Name: conv2d_100
Trainable Parameters Count: 80
Layer Name: conv2d_101
Trainable Parameters Count: 2080
Layer Name: conv2d_102
Trainable Parameters Count: 8256
Layer Name: conv2d_103
Trainable Parameters Count: 32896
Layer Name: output1
Trainable Parameters Count: 258
Layer Name: output2
Trainable Parameters Count: 258
------------------------
Layer Name: conv2d
Trainable Parameters Count: 80
Layer Name: conv2d_1
Trainable Parameters Count: 2080
Layer Name: conv2d_2
Trainable Parameters Count: 8256
Layer Name: conv2d_3
Trainable Parameters Count: 32896
Layer Name: dense
Trainable Parameters Count: 258
------------------------
Transferred weights from layer 'dense' to layer 'output1'.
Transferred weights from layer 'dense' to layer 'output2'.
OUT Gunshot:  [1.421236515045166, 0.1844685971736908, 1.2367676496505737, 0.9414715766906738, 0.5117056965827942]
Training Accuracy for Gunshot:  0.9414715766906738
OUT Gunshot:  [1.331782579421997, 0.19050715863704681, 1.141275405883789, 0.9

#SCREAM VS SIREN ARQ 1 MAX/MIN/AVG VALUE IN LAYER

In [210]:
#1	3	4096	4096	4096	0	2
#2	12	4096	4096	4096	0	3
#3	27	4096	4096	4096	0	3
#4	45	4096	4096	4096	0	5

samplerate = 22050
longitudMaxAudio = 4
Nmfcc = 3
Nfft = 4096
NwinL = 4096
iterableNhopL = 1.0
NhopL =  4096       #int(iterableNhopL*NwinL)
k_size = 2
num_rows = Nmfcc
num_columns = int(samplerate*longitudMaxAudio/NhopL) + int(samplerate*longitudMaxAudio/NhopL*0.05)  #Calculo longitud de salida de mfcc con 5% de tolerancia para longitud de audios
num_channels = 1

num_labels = 2 #4

#model = getModel(num_rows, num_columns, num_channels, num_labels, k_size)
#model = getModelFunctional(num_rows, num_columns, num_channels, num_labels, k_size)
Fusedmodel = getModelFunctionalModified(num_rows, num_columns, num_channels, num_labels, k_size)

# Compila el modelo (es necesario antes de la evaluación)
Fusedmodel.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#TRAINED MODELS TO FUSE
modelScreamTLSave_path_1 = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/scream/saved_scream_TL_1'
modelScreamTL_1 = tf.keras.models.load_model(modelScreamTLSave_path_1)

modelSirenTLSave_path_1 = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/siren/saved_siren_TL_1'
modelSirenTL_1 = tf.keras.models.load_model(modelSirenTLSave_path_1)

#layer_index = 3  # Specify the index of the layer (e.g., 0 for the first convolutional layer)
#change_filters(modelGunshotTL, modelScreamTL, 0, "model1_to_model2")
#change_filters(modelGunshotTL, modelScreamTL, 1, "model1_to_model2")
#change_filters(modelGunshotTL, modelScreamTL, 2, "model2_to_model1")
#change_filters(modelGunshotTL, modelScreamTL, layer_index, "model1_to_model2")

# Iterar sobre las capas y verificar la cantidad de parámetros entrenables
for layer in Fusedmodel.layers:
    trainable_params_count = layer.count_params()
    if trainable_params_count > 0:
        print("Layer Name:", layer.name)
        print("Trainable Parameters Count:", trainable_params_count)
print("------------------------")

# Iterar sobre las capas y verificar la cantidad de parámetros entrenables
for layer in modelScreamTL_1.layers:
    trainable_params_count = layer.count_params()
    if trainable_params_count > 0:
        print("Layer Name:", layer.name)
        print("Trainable Parameters Count:", trainable_params_count)
print("------------------------")


#transfer_parameters_between_models(modelGunshotTL, 0, Fusedmodel, 0)
#transfer_parameters_between_models(modelGunshotTL, 1, Fusedmodel, 1)
#transfer_parameters_between_models(modelScreamTL, 2, Fusedmodel, 2)
#transfer_parameters_between_models(modelGunshotTL, 3, Fusedmodel, 3)



#transfer_max_abs_conv_weights(modelScreamTL_1, modelSirenTL_1, Fusedmodel)
#transfer_min_abs_conv_weights(modelScreamTL_1, modelSirenTL_1, Fusedmodel)
transfer_avg_conv_weights(modelScreamTL_1, modelSirenTL_1, Fusedmodel)

transfer_parameters_between_models(modelScreamTL_1, 4, Fusedmodel, 4)
transfer_parameters_between_models(modelSirenTL_1, 4, Fusedmodel, 5)


#Fusedmodel.summary()


# Evaluating the FUSED model on the training and testing sets

scoreScream = Fusedmodel.evaluate(x_trainScream, [y_trainScream, np.resize(y_trainSiren, (y_trainScream.shape[0], *y_trainSiren.shape[1:]))], verbose=0)
print("OUT Scream: ", scoreScream)
print("Training Accuracy for Scream: ", scoreScream[3])

scoreScream = Fusedmodel.evaluate(x_testScream, [y_testScream, np.resize(y_testSiren, (y_testScream.shape[0], *y_testSiren.shape[1:]))], verbose=0)
print("OUT Scream: ", scoreScream)
print("Testing Accuracy for Scream: ", scoreScream[3])



scoreSiren= Fusedmodel.evaluate(x_trainSiren, [np.resize(y_trainScream, (y_trainSiren.shape[0], *y_trainScream.shape[1:])), y_trainSiren], verbose=0)
print("OUT Siren: ", scoreSiren)
print("Training Accuracy for Siren: ", scoreSiren[4])

scoreSiren= Fusedmodel.evaluate(x_testSiren, [np.resize(y_testScream, (y_testSiren.shape[0], *y_testScream.shape[1:])), y_testSiren], verbose=0)
print("OUT Siren: ", scoreSiren)
print("Testing Accuracy for Siren: ", scoreSiren[4])


y_true = np.argmax(y_testScream,axis=1)
y_pred = np.argmax(Fusedmodel.predict(x_testScream)[0],axis=1)
print('\nConfusion Matrix SCREAM:\n\n')
print('SHAPEEE : ',y_true.shape)
print('SHAPEEE : ',y_pred.shape)
print(confusion_matrix(y_true,y_pred))
print('\n\nClassification Report : \n\n',classification_report(y_true,y_pred))

y_true = np.argmax(y_testSiren,axis=1)
y_pred = np.argmax(Fusedmodel.predict(x_testSiren)[1],axis=1)
print('\nConfusion Matrix SIREN:\n\n')
print('SHAPEEE : ',y_true.shape)
print('SHAPEEE : ',y_pred.shape)
print(confusion_matrix(y_true,y_pred))
print('\n\nClassification Report : \n\n',classification_report(y_true,y_pred))

Layer Name: conv2d_112
Trainable Parameters Count: 80
Layer Name: conv2d_113
Trainable Parameters Count: 2080
Layer Name: conv2d_114
Trainable Parameters Count: 8256
Layer Name: conv2d_115
Trainable Parameters Count: 32896
Layer Name: output1
Trainable Parameters Count: 258
Layer Name: output2
Trainable Parameters Count: 258
------------------------
Layer Name: conv2d
Trainable Parameters Count: 80
Layer Name: conv2d_1
Trainable Parameters Count: 2080
Layer Name: conv2d_2
Trainable Parameters Count: 8256
Layer Name: conv2d_3
Trainable Parameters Count: 32896
Layer Name: dense
Trainable Parameters Count: 258
------------------------
Transferred weights from layer 'dense' to layer 'output1'.
Transferred weights from layer 'dense' to layer 'output2'.
OUT Scream:  [1.0257936716079712, 0.08844684809446335, 0.9373467564582825, 0.984000027179718, 0.5216000080108643]
Training Accuracy for Scream:  0.984000027179718
OUT Scream:  [0.9769054651260376, 0.10497266054153442, 0.8719328045845032, 0.97

#GUN VS SCREAM ARQ 4 MAX/MIN/AVG VALUE IN LAYER

In [159]:
#1	3	4096	4096	4096	0	2
#2	12	4096	4096	4096	0	3
#3	27	4096	4096	4096	0	3
#4	45	4096	4096	4096	0	5

samplerate = 22050
longitudMaxAudio = 4
Nmfcc = 45
Nfft = 4096
NwinL = 4096
iterableNhopL = 1.0
NhopL =  4096       #int(iterableNhopL*NwinL)
k_size = 5
num_rows = Nmfcc
num_columns = int(samplerate*longitudMaxAudio/NhopL) + int(samplerate*longitudMaxAudio/NhopL*0.05)  #Calculo longitud de salida de mfcc con 5% de tolerancia para longitud de audios
num_channels = 1

num_labels = 2 #4

#model = getModel(num_rows, num_columns, num_channels, num_labels, k_size)
#model = getModelFunctional(num_rows, num_columns, num_channels, num_labels, k_size)
Fusedmodel = getModelFunctionalModified(num_rows, num_columns, num_channels, num_labels, k_size)

# Compila el modelo (es necesario antes de la evaluación)
Fusedmodel.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#TRAINED MODELS TO FUSE
modelGunshotTLSave_path_4 = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/gunshot/saved_gunshot_TL_4'
modelGunshotTL_4 = tf.keras.models.load_model(modelGunshotTLSave_path_4)

modelScreamTLSave_path_4 = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/scream/saved_scream_TL_4'
modelScreamTL_4 = tf.keras.models.load_model(modelScreamTLSave_path_4)

#layer_index = 3  # Specify the index of the layer (e.g., 0 for the first convolutional layer)
#change_filters(modelGunshotTL, modelScreamTL, 0, "model1_to_model2")
#change_filters(modelGunshotTL, modelScreamTL, 1, "model1_to_model2")
#change_filters(modelGunshotTL, modelScreamTL, 2, "model2_to_model1")
#change_filters(modelGunshotTL, modelScreamTL, layer_index, "model1_to_model2")

# Iterar sobre las capas y verificar la cantidad de parámetros entrenables
for layer in Fusedmodel.layers:
    trainable_params_count = layer.count_params()
    if trainable_params_count > 0:
        print("Layer Name:", layer.name)
        print("Trainable Parameters Count:", trainable_params_count)
print("------------------------")

# Iterar sobre las capas y verificar la cantidad de parámetros entrenables
for layer in modelGunshotTL_4.layers:
    trainable_params_count = layer.count_params()
    if trainable_params_count > 0:
        print("Layer Name:", layer.name)
        print("Trainable Parameters Count:", trainable_params_count)
print("------------------------")


#transfer_parameters_between_models(modelGunshotTL, 0, Fusedmodel, 0)
#transfer_parameters_between_models(modelGunshotTL, 1, Fusedmodel, 1)
#transfer_parameters_between_models(modelScreamTL, 2, Fusedmodel, 2)
#transfer_parameters_between_models(modelGunshotTL, 3, Fusedmodel, 3)



#transfer_max_abs_conv_weights(modelGunshotTL_4, modelScreamTL_4, Fusedmodel)
#transfer_min_abs_conv_weights(modelGunshotTL_4, modelScreamTL_4, Fusedmodel)
transfer_avg_conv_weights(modelGunshotTL_4, modelScreamTL_4, Fusedmodel)

transfer_parameters_between_models(modelGunshotTL_4, 4, Fusedmodel, 4)
transfer_parameters_between_models(modelScreamTL_4, 4, Fusedmodel, 5)


#Fusedmodel.summary()


# Evaluating the FUSED model on the training and testing sets

scoreGunshot = Fusedmodel.evaluate(x_trainGunshot, [y_trainGunshot, np.resize(y_trainScream, (y_trainGunshot.shape[0], *y_trainScream.shape[1:]))], verbose=0)
print("OUT Gunshot: ", scoreGunshot)
print("Training Accuracy for Gunshot: ", scoreGunshot[3])

scoreGunshot = Fusedmodel.evaluate(x_testGunshot, [y_testGunshot, np.resize(y_testScream, (y_testGunshot.shape[0], *y_testScream.shape[1:]))], verbose=0)
print("OUT Gunshot: ", scoreGunshot)
print("Testing Accuracy for Gunshot: ", scoreGunshot[3])



scoreScream= Fusedmodel.evaluate(x_trainScream, [np.resize(y_trainGunshot, (y_trainScream.shape[0], *y_trainGunshot.shape[1:])), y_trainScream], verbose=0)
print("OUT Scream: ", scoreScream)
print("Training Accuracy for Scream: ", scoreScream[4])

scoreScream= Fusedmodel.evaluate(x_testScream, [np.resize(y_testGunshot, (y_testScream.shape[0], *y_testGunshot.shape[1:])), y_testScream], verbose=0)
print("OUT Scream: ", scoreScream)
print("Testing Accuracy for Scream: ", scoreScream[4])


y_true = np.argmax(y_testGunshot,axis=1)
y_pred = np.argmax(Fusedmodel.predict(x_testGunshot)[0],axis=1)
print('\nConfusion Matrix GUNSHOT:\n\n')
print('SHAPEEE : ',y_true.shape)
print('SHAPEEE : ',y_pred.shape)
print(confusion_matrix(y_true,y_pred))
print('\n\nClassification Report : \n\n',classification_report(y_true,y_pred))

y_true = np.argmax(y_testScream,axis=1)
y_pred = np.argmax(Fusedmodel.predict(x_testScream)[1],axis=1)
print('\nConfusion Matrix SCREAM:\n\n')
print('SHAPEEE : ',y_true.shape)
print('SHAPEEE : ',y_pred.shape)
print(confusion_matrix(y_true,y_pred))
print('\n\nClassification Report : \n\n',classification_report(y_true,y_pred))

Layer Name: conv2d_52
Trainable Parameters Count: 416
Layer Name: conv2d_53
Trainable Parameters Count: 12832
Layer Name: conv2d_54
Trainable Parameters Count: 51264
Layer Name: conv2d_55
Trainable Parameters Count: 204928
Layer Name: output1
Trainable Parameters Count: 258
Layer Name: output2
Trainable Parameters Count: 258
------------------------
Layer Name: conv2d_12
Trainable Parameters Count: 416
Layer Name: conv2d_13
Trainable Parameters Count: 12832
Layer Name: conv2d_14
Trainable Parameters Count: 51264
Layer Name: conv2d_15
Trainable Parameters Count: 204928
Layer Name: dense_3
Trainable Parameters Count: 258
------------------------
Transferred weights from layer 'dense_3' to layer 'output1'.
Transferred weights from layer 'dense_3' to layer 'output2'.
OUT Gunshot:  [2.6380538940429688, 0.14506328105926514, 2.492990255355835, 0.9448160529136658, 0.5050167441368103]
Training Accuracy for Gunshot:  0.9448160529136658
OUT Gunshot:  [2.4106714725494385, 0.10860927402973175, 2.30

#GUN VS SIREN ARQ 4 MAX/MIN/AVG VALUE IN LAYER

In [163]:
#1	3	4096	4096	4096	0	2
#2	12	4096	4096	4096	0	3
#3	27	4096	4096	4096	0	3
#4	45	4096	4096	4096	0	5

samplerate = 22050
longitudMaxAudio = 4
Nmfcc = 45
Nfft = 4096
NwinL = 4096
iterableNhopL = 1.0
NhopL =  4096       #int(iterableNhopL*NwinL)
k_size = 5
num_rows = Nmfcc
num_columns = int(samplerate*longitudMaxAudio/NhopL) + int(samplerate*longitudMaxAudio/NhopL*0.05)  #Calculo longitud de salida de mfcc con 5% de tolerancia para longitud de audios
num_channels = 1

num_labels = 2 #4

#model = getModel(num_rows, num_columns, num_channels, num_labels, k_size)
#model = getModelFunctional(num_rows, num_columns, num_channels, num_labels, k_size)
Fusedmodel = getModelFunctionalModified(num_rows, num_columns, num_channels, num_labels, k_size)

# Compila el modelo (es necesario antes de la evaluación)
Fusedmodel.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#TRAINED MODELS TO FUSE
modelGunshotTLSave_path_4 = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/gunshot/saved_gunshot_TL_4'
modelGunshotTL_4 = tf.keras.models.load_model(modelGunshotTLSave_path_4)

modelSirenTLSave_path_4 = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/siren/saved_siren_TL_4'
modelSirenTL_4 = tf.keras.models.load_model(modelSirenTLSave_path_4)

#layer_index = 3  # Specify the index of the layer (e.g., 0 for the first convolutional layer)
#change_filters(modelGunshotTL, modelScreamTL, 0, "model1_to_model2")
#change_filters(modelGunshotTL, modelScreamTL, 1, "model1_to_model2")
#change_filters(modelGunshotTL, modelScreamTL, 2, "model2_to_model1")
#change_filters(modelGunshotTL, modelScreamTL, layer_index, "model1_to_model2")

# Iterar sobre las capas y verificar la cantidad de parámetros entrenables
for layer in Fusedmodel.layers:
    trainable_params_count = layer.count_params()
    if trainable_params_count > 0:
        print("Layer Name:", layer.name)
        print("Trainable Parameters Count:", trainable_params_count)
print("------------------------")

# Iterar sobre las capas y verificar la cantidad de parámetros entrenables
for layer in modelGunshotTL_4.layers:
    trainable_params_count = layer.count_params()
    if trainable_params_count > 0:
        print("Layer Name:", layer.name)
        print("Trainable Parameters Count:", trainable_params_count)
print("------------------------")


#transfer_parameters_between_models(modelGunshotTL, 0, Fusedmodel, 0)
#transfer_parameters_between_models(modelGunshotTL, 1, Fusedmodel, 1)
#transfer_parameters_between_models(modelScreamTL, 2, Fusedmodel, 2)
#transfer_parameters_between_models(modelGunshotTL, 3, Fusedmodel, 3)



#transfer_max_abs_conv_weights(modelGunshotTL_4, modelSirenTL_4, Fusedmodel)
#transfer_min_abs_conv_weights(modelGunshotTL_4, modelSirenTL_4, Fusedmodel)
transfer_avg_conv_weights(modelGunshotTL_4, modelSirenTL_4, Fusedmodel)

transfer_parameters_between_models(modelGunshotTL_4, 4, Fusedmodel, 4)
transfer_parameters_between_models(modelSirenTL_4, 4, Fusedmodel, 5)


#Fusedmodel.summary()


# Evaluating the FUSED model on the training and testing sets

scoreGunshot = Fusedmodel.evaluate(x_trainGunshot, [y_trainGunshot, np.resize(y_trainSiren, (y_trainGunshot.shape[0], *y_trainSiren.shape[1:]))], verbose=0)
print("OUT Gunshot: ", scoreGunshot)
print("Training Accuracy for Gunshot: ", scoreGunshot[3])

scoreGunshot = Fusedmodel.evaluate(x_testGunshot, [y_testGunshot, np.resize(y_testSiren, (y_testGunshot.shape[0], *y_testSiren.shape[1:]))], verbose=0)
print("OUT Gunshot: ", scoreGunshot)
print("Testing Accuracy for Gunshot: ", scoreGunshot[3])



scoreSiren= Fusedmodel.evaluate(x_trainSiren, [np.resize(y_trainGunshot, (y_trainSiren.shape[0], *y_trainGunshot.shape[1:])), y_trainSiren], verbose=0)
print("OUT Siren: ", scoreSiren)
print("Training Accuracy for Siren: ", scoreSiren[4])

scoreSiren= Fusedmodel.evaluate(x_testSiren, [np.resize(y_testGunshot, (y_testSiren.shape[0], *y_testGunshot.shape[1:])), y_testSiren], verbose=0)
print("OUT Siren: ", scoreSiren)
print("Testing Accuracy for Siren: ", scoreSiren[4])


y_true = np.argmax(y_testGunshot,axis=1)
y_pred = np.argmax(Fusedmodel.predict(x_testGunshot)[0],axis=1)
print('\nConfusion Matrix GUNSHOT:\n\n')
print('SHAPEEE : ',y_true.shape)
print('SHAPEEE : ',y_pred.shape)
print(confusion_matrix(y_true,y_pred))
print('\n\nClassification Report : \n\n',classification_report(y_true,y_pred))

y_true = np.argmax(y_testSiren,axis=1)
y_pred = np.argmax(Fusedmodel.predict(x_testSiren)[1],axis=1)
print('\nConfusion Matrix SIREN:\n\n')
print('SHAPEEE : ',y_true.shape)
print('SHAPEEE : ',y_pred.shape)
print(confusion_matrix(y_true,y_pred))
print('\n\nClassification Report : \n\n',classification_report(y_true,y_pred))

Layer Name: conv2d_64
Trainable Parameters Count: 416
Layer Name: conv2d_65
Trainable Parameters Count: 12832
Layer Name: conv2d_66
Trainable Parameters Count: 51264
Layer Name: conv2d_67
Trainable Parameters Count: 204928
Layer Name: output1
Trainable Parameters Count: 258
Layer Name: output2
Trainable Parameters Count: 258
------------------------
Layer Name: conv2d_12
Trainable Parameters Count: 416
Layer Name: conv2d_13
Trainable Parameters Count: 12832
Layer Name: conv2d_14
Trainable Parameters Count: 51264
Layer Name: conv2d_15
Trainable Parameters Count: 204928
Layer Name: dense_3
Trainable Parameters Count: 258
------------------------
Transferred weights from layer 'dense_3' to layer 'output1'.
Transferred weights from layer 'dense_3' to layer 'output2'.
OUT Gunshot:  [2.002363681793213, 0.13800734281539917, 1.8643566370010376, 0.9448160529136658, 0.49665552377700806]
Training Accuracy for Gunshot:  0.9448160529136658
OUT Gunshot:  [1.6107232570648193, 0.1036984995007515, 1.50

#SCREAM VS SIREN ARQ 4 MAX/MIN/AVG VALUE IN LAYER

In [167]:
#1	3	4096	4096	4096	0	2
#2	12	4096	4096	4096	0	3
#3	27	4096	4096	4096	0	3
#4	45	4096	4096	4096	0	5

samplerate = 22050
longitudMaxAudio = 4
Nmfcc = 45
Nfft = 4096
NwinL = 4096
iterableNhopL = 1.0
NhopL =  4096       #int(iterableNhopL*NwinL)
k_size = 5
num_rows = Nmfcc
num_columns = int(samplerate*longitudMaxAudio/NhopL) + int(samplerate*longitudMaxAudio/NhopL*0.05)  #Calculo longitud de salida de mfcc con 5% de tolerancia para longitud de audios
num_channels = 1

num_labels = 2 #4

#model = getModel(num_rows, num_columns, num_channels, num_labels, k_size)
#model = getModelFunctional(num_rows, num_columns, num_channels, num_labels, k_size)
Fusedmodel = getModelFunctionalModified(num_rows, num_columns, num_channels, num_labels, k_size)

# Compila el modelo (es necesario antes de la evaluación)
Fusedmodel.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#TRAINED MODELS TO FUSE
modelScreamTLSave_path_4 = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/scream/saved_scream_TL_4'
modelScreamTL_4 = tf.keras.models.load_model(modelScreamTLSave_path_4)

modelSirenTLSave_path_4 = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/siren/saved_siren_TL_4'
modelSirenTL_4 = tf.keras.models.load_model(modelSirenTLSave_path_4)

#layer_index = 3  # Specify the index of the layer (e.g., 0 for the first convolutional layer)
#change_filters(modelGunshotTL, modelScreamTL, 0, "model1_to_model2")
#change_filters(modelGunshotTL, modelScreamTL, 1, "model1_to_model2")
#change_filters(modelGunshotTL, modelScreamTL, 2, "model2_to_model1")
#change_filters(modelGunshotTL, modelScreamTL, layer_index, "model1_to_model2")

# Iterar sobre las capas y verificar la cantidad de parámetros entrenables
for layer in Fusedmodel.layers:
    trainable_params_count = layer.count_params()
    if trainable_params_count > 0:
        print("Layer Name:", layer.name)
        print("Trainable Parameters Count:", trainable_params_count)
print("------------------------")

# Iterar sobre las capas y verificar la cantidad de parámetros entrenables
for layer in modelScreamTL_4.layers:
    trainable_params_count = layer.count_params()
    if trainable_params_count > 0:
        print("Layer Name:", layer.name)
        print("Trainable Parameters Count:", trainable_params_count)
print("------------------------")


#transfer_parameters_between_models(modelGunshotTL, 0, Fusedmodel, 0)
#transfer_parameters_between_models(modelGunshotTL, 1, Fusedmodel, 1)
#transfer_parameters_between_models(modelScreamTL, 2, Fusedmodel, 2)
#transfer_parameters_between_models(modelGunshotTL, 3, Fusedmodel, 3)



#transfer_max_abs_conv_weights(modelScreamTL_4, modelSirenTL_4, Fusedmodel)
#transfer_min_abs_conv_weights(modelScreamTL_4, modelSirenTL_4, Fusedmodel)
transfer_avg_conv_weights(modelScreamTL_4, modelSirenTL_4, Fusedmodel)

transfer_parameters_between_models(modelScreamTL_4, 4, Fusedmodel, 4)
transfer_parameters_between_models(modelSirenTL_4, 4, Fusedmodel, 5)


#Fusedmodel.summary()


# Evaluating the FUSED model on the training and testing sets

scoreScream = Fusedmodel.evaluate(x_trainScream, [y_trainScream, np.resize(y_trainSiren, (y_trainScream.shape[0], *y_trainSiren.shape[1:]))], verbose=0)
print("OUT Scream: ", scoreScream)
print("Training Accuracy for Scream: ", scoreScream[3])

scoreScream = Fusedmodel.evaluate(x_testScream, [y_testScream, np.resize(y_testSiren, (y_testScream.shape[0], *y_testSiren.shape[1:]))], verbose=0)
print("OUT Scream: ", scoreScream)
print("Testing Accuracy for Scream: ", scoreScream[3])



scoreSiren= Fusedmodel.evaluate(x_trainSiren, [np.resize(y_trainScream, (y_trainSiren.shape[0], *y_trainScream.shape[1:])), y_trainSiren], verbose=0)
print("OUT Siren: ", scoreSiren)
print("Training Accuracy for Siren: ", scoreSiren[4])

scoreSiren= Fusedmodel.evaluate(x_testSiren, [np.resize(y_testScream, (y_testSiren.shape[0], *y_testScream.shape[1:])), y_testSiren], verbose=0)
print("OUT Siren: ", scoreSiren)
print("Testing Accuracy for Siren: ", scoreSiren[4])


y_true = np.argmax(y_testScream,axis=1)
y_pred = np.argmax(Fusedmodel.predict(x_testScream)[0],axis=1)
print('\nConfusion Matrix SCREAM:\n\n')
print('SHAPEEE : ',y_true.shape)
print('SHAPEEE : ',y_pred.shape)
print(confusion_matrix(y_true,y_pred))
print('\n\nClassification Report : \n\n',classification_report(y_true,y_pred))

y_true = np.argmax(y_testSiren,axis=1)
y_pred = np.argmax(Fusedmodel.predict(x_testSiren)[1],axis=1)
print('\nConfusion Matrix SIREN:\n\n')
print('SHAPEEE : ',y_true.shape)
print('SHAPEEE : ',y_pred.shape)
print(confusion_matrix(y_true,y_pred))
print('\n\nClassification Report : \n\n',classification_report(y_true,y_pred))

Layer Name: conv2d_76
Trainable Parameters Count: 416
Layer Name: conv2d_77
Trainable Parameters Count: 12832
Layer Name: conv2d_78
Trainable Parameters Count: 51264
Layer Name: conv2d_79
Trainable Parameters Count: 204928
Layer Name: output1
Trainable Parameters Count: 258
Layer Name: output2
Trainable Parameters Count: 258
------------------------
Layer Name: conv2d_12
Trainable Parameters Count: 416
Layer Name: conv2d_13
Trainable Parameters Count: 12832
Layer Name: conv2d_14
Trainable Parameters Count: 51264
Layer Name: conv2d_15
Trainable Parameters Count: 204928
Layer Name: dense_3
Trainable Parameters Count: 258
------------------------
Transferred weights from layer 'dense_3' to layer 'output1'.
Transferred weights from layer 'dense_3' to layer 'output2'.
OUT Scream:  [2.7214295864105225, 0.031180502846837044, 2.690248727798462, 0.9968000054359436, 0.5055999755859375]
Training Accuracy for Scream:  0.9968000054359436
OUT Scream:  [1.99534010887146, 0.0664723739027977, 1.9288678